<a href="https://colab.research.google.com/github/rohansamuel7/Library-Management-System/blob/main/Library_Management_System_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import datetime

class Book:
    def __init__(self, title, author, isbn, quantity):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.quantity = quantity

    def update_details(self, title, author, isbn, quantity):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.quantity = quantity

    def display_details(self):
        print(f"Title: {self.title}, Author: {self.author}, ISBN: {self.isbn}, Quantity: {self.quantity}")

class Patron:
    def __init__(self, name, id, contact_info):
        self.name = name
        self.id = id
        self.contact_info = contact_info

    def update_details(self, name, id, contact_info):
        self.name = name
        self.id = id
        self.contact_info = contact_info

    def display_details(self):
        print(f"Name: {self.name}, ID: {self.id}, Contact Info: {self.contact_info}")

class Transaction:
    def __init__(self, book, patron):
        self.book = book
        self.patron = patron
        self.checkout_date = None
        self.due_date = None
        self.return_date = None

    def check_out(self):
        if self.book.quantity > 0:
            self.book.quantity -= 1
            self.checkout_date = datetime.datetime.now()
            self.due_date = self.checkout_date + datetime.timedelta(days=14)  # Assuming 2 weeks due date
            print(f"Book '{self.book.title}' checked out by {self.patron.name}. Due date: {self.due_date}")
        else:
            print("Sorry, the book is not available for checkout.")

    def check_in(self):
        if self.return_date is None:
            self.book.quantity += 1
            self.return_date = datetime.datetime.now()
            print(f"Book '{self.book.title}' checked in by {self.patron.name}.")
        else:
            print("The book has already been returned.")

    def display_details(self):
        if self.checkout_date:
            print(f"Book: {self.book.title}, Patron: {self.patron.name}, Checkout Date: {self.checkout_date}")
            if self.due_date:
                print(f"Due Date: {self.due_date}")
            if self.return_date:
                print(f"Return Date: {self.return_date}")
        else:
            print("No transaction details available.")

class Library:
    def __init__(self):
        self.books = []
        self.patrons = []
        self.transactions = []

        self.load_data()

    def load_data(self):
        self.load_from_json("books.json", Book, self.books)
        self.load_from_json("patrons.json", Patron, self.patrons)

    def load_from_json(self, filename, class_type, collection):
        try:
            with open(filename, "r") as f:
                data = json.load(f)
                for item in data:
                    obj = class_type(**item)
                    collection.append(obj)
        except FileNotFoundError:
            pass

    def save_data(self):
        self.save_to_json("books.json", self.books)
        self.save_to_json("patrons.json", self.patrons)

    def save_to_json(self, filename, collection):
        with open(filename, "w") as f:
            data = [obj.__dict__ for obj in collection]
            json.dump(data, f, indent=4)

    def add_book(self, book):
        self.books.append(book)

    def remove_book(self, isbn):
        for book in self.books:
            if book.isbn == isbn:
                self.books.remove(book)
                break
        else:
            print("Book not found!")

    def add_patron(self, patron):
        self.patrons.append(patron)

    def remove_patron(self, id):
        for patron in self.patrons:
            if patron.id == id:
                self.patrons.remove(patron)
                break
        else:
            print("Patron not found!")

    def search_books(self, title=None, author=None, isbn=None):
        result = []
        for book in self.books:
            if (not title or title.lower() in book.title.lower()) and \
               (not author or author.lower() in book.author.lower()) and \
               (not isbn or isbn == book.isbn):
                result.append(book)
        return result

    def display_books(self, books=None):
        if not books:
            books = self.books
        print("Books in the library:")
        for book in books:
            book.display_details()

    def display_patrons(self):
        print("Patrons of the library:")
        for patron in self.patrons:
            patron.display_details()

    def check_out_book(self, book, patron):
        if book.quantity > 0:
            book.quantity -= 1
            transaction = Transaction(book, patron)
            self.transactions.append(transaction)
            print(f"Book '{book.title}' checked out by {patron.name}.")
        else:
            print("Sorry, the book is not available for checkout.")

    def check_in_book(self, book, patron):
        for transaction in self.transactions:
            if transaction.book == book and transaction.patron == patron:
                transaction.check_in()
                break
        else:
            print("No transaction found for this book and patron.")

def main():
    library = Library()

    while True:
        print("\nLibrary Management System")
        print("1. Add book")
        print("2. Remove book")
        print("3. Add patron")
        print("4. Remove patron")
        print("5. Search books")
        print("6. Display books")
        print("7. Display patrons")
        print("8. Check out book")
        print("9. Check in book")
        print("10. Save data")
        print("11. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            isbn = input("Enter book ISBN: ")
            quantity = int(input("Enter book quantity: "))
            library.add_book(Book(title, author, isbn, quantity))
        elif choice == "2":
            isbn = input("Enter book ISBN: ")
            library.remove_book(isbn)
        elif choice == "3":
            name = input("Enter patron name: ")
            id = input("Enter patron ID: ")
            contact_info = input("Enter patron contact info: ")
            library.add_patron(Patron(name, id, contact_info))
        elif choice == "4":
            id = input("Enter patron ID: ")
            library.remove_patron(id)
        elif choice == "5":
            title = input("Enter book title (leave blank to skip): ").strip()
            author = input("Enter book author (leave blank to skip): ").strip()
            isbn = input("Enter book ISBN (leave blank to skip): ").strip()
            result = library.search_books(title, author, isbn)
            library.display_books(result)
        elif choice == "6":
            library.display_books()
        elif choice == "7":
            library.display_patrons()
        elif choice == "8":
            isbn = input("Enter book ISBN: ")
            patron_id = input("Enter patron ID: ")
            book = next((b for b in library.books if b.isbn == isbn), None)
            patron = next((p for p in library.patrons if p.id == patron_id), None)
            if book and patron:
                library.check_out_book(book, patron)
            else:
                print("Book or patron not found.")
        elif choice == "9":
            isbn = input("Enter book ISBN: ")
            patron_id = input("Enter patron ID: ")
            book = next((b for b in library.books if b.isbn == isbn), None)
            patron = next((p for p in library.patrons if p.id == patron_id), None)
            if book and patron:
                library.check_in_book(book, patron)
            else:
                print("Book or patron not found.")
        elif choice == "10":
            library.save_data()
            print("Data saved successfully.")
        elif choice == "11":
            print("Exiting...")
            break
        else:
            print("Invalid choice")

if __name__ == "__main__":
    main()



Library Management System
1. Add book
2. Remove book
3. Add patron
4. Remove patron
5. Search books
6. Display books
7. Display patrons
8. Check out book
9. Check in book
10. Save data
11. Exit
Enter your choice: 11
Exiting...
